In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import torch
import torchtuples as tt

from pycox.preprocessing.label_transforms import LabTransDiscreteTime
from pycox.models import DeepHit
from pycox.evaluation import EvalSurv


np.random.seed(1234)
_ = torch.manual_seed(1234)


df_train = pd.read_csv("Melanoma_train_data_py.csv")
df_test = pd.read_csv("Melanoma_test_data_py.csv")

## Dealing with Missing values

#df_train['meal.cal'].fillna(df_train['meal.cal'].mean(), inplace=True)
#df_train['wt.loss'].fillna(df_train['wt.loss'].mean(), inplace=True)
#df_test['meal.cal'].fillna(df_test['meal.cal'].mean(), inplace=True)
#df_test['wt.loss'].fillna(df_test['wt.loss'].mean(), inplace=True)


df_train = df_train.dropna()
df_test = df_test.dropna()


get_x = lambda df: (df.drop(columns=['time', 'status']).values.astype('float32'))
x_train = get_x(df_train)
x_test = get_x(df_test)


class LabTransform(LabTransDiscreteTime):
    def transform(self, durations, events):
        durations, is_event = super().transform(durations, events > 0)
        events[is_event == 0] = 0
        return durations, events.astype('int64')


num_durations = 108
labtrans = LabTransform(num_durations)
get_target = lambda df: (df['time'].values, df['status'].values)


y_train = labtrans.fit_transform(*get_target(df_train))
y_test = labtrans.fit_transform(*get_target(df_test))
durations_test, events_test = get_target(df_test)


In [2]:
class CauseSpecificNet(torch.nn.Module):
    """Network structure similar to the DeepHit paper, but without the residual
    connections (for simplicity).
    """
    def __init__(self, in_features, num_nodes_shared, num_nodes_indiv, num_risks,
                 out_features, batch_norm=True, dropout=None):
        super().__init__()
        self.shared_net = tt.practical.MLPVanilla(
            in_features, num_nodes_shared[:-1], num_nodes_shared[-1],
            batch_norm, dropout,
        )
        self.risk_nets = torch.nn.ModuleList()
        for _ in range(num_risks):
            net = tt.practical.MLPVanilla(
                num_nodes_shared[-1], num_nodes_indiv, out_features,
                batch_norm, dropout,
            )
            self.risk_nets.append(net)
    def forward(self, input):
        out = self.shared_net(input)
        out = [net(out) for net in self.risk_nets]
        out = torch.stack(out, dim=1)
        return out


in_features = x_train.shape[1]
num_risks = y_train[1].max()
out_features = len(labtrans.cuts)
dropout = [0.0]



list_num_nodes_shared = [[32, 8], [16, 8], [16, 4], [8, 4]]
list_num_nodes_indiv = [[32], [16], [8]]
list_batch_norm = [False, True]
list_lr = [0.1, 0.01, 0.001, 0.0001]
list_alpha = [0.1, 0.2, 0.3, 0.4]
list_sigma = [0.1, 0.2, 0.3, 0.4]
list_batch_size = [128, 256]


parameters = []
for num_nodes_shared in list_num_nodes_shared:
    for num_nodes_indiv in list_num_nodes_indiv:
        for batch_norm in list_batch_norm:
            for lr in list_lr:
                for alpha in list_alpha:
                    for sigma in list_sigma:
                        for batch_size in list_batch_size:
                            parameters.append([num_nodes_shared, num_nodes_indiv, batch_norm, lr, alpha, sigma, batch_size])



deephit_cv_results = pd.DataFrame(parameters)
deephit_cv_results["cindex"] = 0

In [ ]:
kf = KFold(n_splits = 5)


for index in range(deephit_cv_results.shape[0]):
    num_nodes_shared = deephit_cv_results.iloc[index, 0]
    num_nodes_indiv = deephit_cv_results.iloc[index, 1]
    batch_norm = deephit_cv_results.iloc[index, 2]
    lr = deephit_cv_results.iloc[index, 3]
    alpha = deephit_cv_results.iloc[index, 4]
    sigma = deephit_cv_results.iloc[index, 5]
    batch_size = deephit_cv_results.iloc[index, 6]
    
    cindexes = []
    for train_index, test_index in kf.split(df_train):
        X_tr = x_train[train_index, ]
        X_val = x_train[test_index, ]
        Y_tr_0 = y_train[0][train_index, ]
        Y_tr_1 = y_train[1][train_index, ]
        Y_val_0 = y_train[0][test_index, ]
        Y_val_1 = y_train[1][test_index, ]
        Y_tr = (Y_tr_0, Y_tr_1)
        Y_val = (Y_val_0, Y_val_1)
        
        # net = SimpleMLP(in_features, num_nodes_shared, num_risks, out_features)
        net = CauseSpecificNet(in_features, num_nodes_shared, num_nodes_indiv, 
                               num_risks, out_features, batch_norm, dropout)
        optimizer = tt.optim.AdamWR(lr = lr, decoupled_weight_decay = 0.01,
                            cycle_eta_multiplier=0.8)
        model = DeepHit(net, optimizer, alpha = alpha, sigma = sigma,
                        duration_index = labtrans.cuts)

        epochs = 100
        callbacks = [tt.callbacks.EarlyStoppingCycle()]
        verbose = False # set to True if you want printout

        log = model.fit(X_tr, Y_tr, int(batch_size), epochs,
                callbacks, verbose, val_data = (X_val, Y_val))
        
        cif = model.predict_cif(x_test)
        cif1 = pd.DataFrame(cif[0], model.duration_index)
        ev1 = EvalSurv(1-cif1, durations_test, events_test == 1, censor_surv='km')
        c_index = ev1.concordance_td()

        cindexes.append(c_index)

    deephit_cv_results.iloc[index, 7] = np.mean(cindexes)
    deephit_cv_results.to_csv('cv.results.deephit.csv', index = False)
    print(deephit_cv_results.iloc[index, ].values)

C:\Users\USR\anaconda3\lib\site-packages\torchtuples\callbacks.py:607: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1174.)
  p.data = p.data.add(-weight_decay * eta, p.data)


[list([32, 8]) list([32]) False 0.1 0.1 0.1 128 0.5012465373961219]
[list([32, 8]) list([32]) False 0.1 0.1 0.1 256 0.4930747922437673]
[list([32, 8]) list([32]) False 0.1 0.1 0.2 128 0.49556786703601113]
[list([32, 8]) list([32]) False 0.1 0.1 0.2 256 0.495983379501385]
[list([32, 8]) list([32]) False 0.1 0.1 0.3 128 0.49127423822714683]
[list([32, 8]) list([32]) False 0.1 0.1 0.3 256 0.5]
[list([32, 8]) list([32]) False 0.1 0.1 0.4 128 0.48462603878116345]
[list([32, 8]) list([32]) False 0.1 0.1 0.4 256 0.4962603878116344]
[list([32, 8]) list([32]) False 0.1 0.2 0.1 128 0.5160664819944598]
[list([32, 8]) list([32]) False 0.1 0.2 0.1 256 0.4630193905817174]
[list([32, 8]) list([32]) False 0.1 0.2 0.2 128 0.47770083102493077]
[list([32, 8]) list([32]) False 0.1 0.2 0.2 256 0.4554016620498615]
[list([32, 8]) list([32]) False 0.1 0.2 0.3 128 0.44806094182825484]
[list([32, 8]) list([32]) False 0.1 0.2 0.3 256 0.47105263157894733]
[list([32, 8]) list([32]) False 0.1 0.2 0.4 128 0.46454293

[list([32, 8]) list([32]) False 0.0001 0.3 0.4 256 0.37839335180055406]
[list([32, 8]) list([32]) False 0.0001 0.4 0.1 128 0.43337950138504155]
[list([32, 8]) list([32]) False 0.0001 0.4 0.1 256 0.4296398891966759]
[list([32, 8]) list([32]) False 0.0001 0.4 0.2 128 0.4121883656509695]
[list([32, 8]) list([32]) False 0.0001 0.4 0.2 256 0.4872576177285318]
[list([32, 8]) list([32]) False 0.0001 0.4 0.3 128 0.37506925207756237]
[list([32, 8]) list([32]) False 0.0001 0.4 0.3 256 0.3745152354570637]
[list([32, 8]) list([32]) False 0.0001 0.4 0.4 128 0.4742382271468145]
[list([32, 8]) list([32]) False 0.0001 0.4 0.4 256 0.5349030470914128]
[list([32, 8]) list([32]) True 0.1 0.1 0.1 128 0.5396121883656508]
[list([32, 8]) list([32]) True 0.1 0.1 0.1 256 0.48614958448753465]
[list([32, 8]) list([32]) True 0.1 0.1 0.2 128 0.564819944598338]
[list([32, 8]) list([32]) True 0.1 0.1 0.2 256 0.6016620498614959]
[list([32, 8]) list([32]) True 0.1 0.1 0.3 128 0.5975069252077562]
[list([32, 8]) list([32

[list([32, 8]) list([32]) True 0.0001 0.2 0.4 256 0.5188365650969529]
[list([32, 8]) list([32]) True 0.0001 0.3 0.1 128 0.46980609418282543]
[list([32, 8]) list([32]) True 0.0001 0.3 0.1 256 0.5246537396121884]
[list([32, 8]) list([32]) True 0.0001 0.3 0.2 128 0.5045706371191137]
[list([32, 8]) list([32]) True 0.0001 0.3 0.2 256 0.43185595567867036]
[list([32, 8]) list([32]) True 0.0001 0.3 0.3 128 0.461218836565097]
[list([32, 8]) list([32]) True 0.0001 0.3 0.3 256 0.5199445983379501]
[list([32, 8]) list([32]) True 0.0001 0.3 0.4 128 0.5518005540166204]
[list([32, 8]) list([32]) True 0.0001 0.3 0.4 256 0.5141274238227147]
[list([32, 8]) list([32]) True 0.0001 0.4 0.1 128 0.5925207756232688]
[list([32, 8]) list([32]) True 0.0001 0.4 0.1 256 0.49030470914127433]
[list([32, 8]) list([32]) True 0.0001 0.4 0.2 128 0.5277008310249307]
[list([32, 8]) list([32]) True 0.0001 0.4 0.2 256 0.4933518005540166]
[list([32, 8]) list([32]) True 0.0001 0.4 0.3 128 0.46786703601108026]
[list([32, 8]) li

[list([32, 8]) list([16]) False 0.0001 0.1 0.4 128 0.35540166204986146]
[list([32, 8]) list([16]) False 0.0001 0.1 0.4 256 0.4473684210526316]
[list([32, 8]) list([16]) False 0.0001 0.2 0.1 128 0.4191135734072022]
[list([32, 8]) list([16]) False 0.0001 0.2 0.1 256 0.42936288088642655]
[list([32, 8]) list([16]) False 0.0001 0.2 0.2 128 0.3850415512465374]
[list([32, 8]) list([16]) False 0.0001 0.2 0.2 256 0.4764542936288089]
[list([32, 8]) list([16]) False 0.0001 0.2 0.3 128 0.4354570637119114]
[list([32, 8]) list([16]) False 0.0001 0.2 0.3 256 0.41855955678670365]
[list([32, 8]) list([16]) False 0.0001 0.2 0.4 128 0.37493074792243763]
[list([32, 8]) list([16]) False 0.0001 0.2 0.4 256 0.39349030470914126]
[list([32, 8]) list([16]) False 0.0001 0.3 0.1 128 0.3986149584487535]
[list([32, 8]) list([16]) False 0.0001 0.3 0.1 256 0.40055401662049867]
[list([32, 8]) list([16]) False 0.0001 0.3 0.2 128 0.41329639889196673]
[list([32, 8]) list([16]) False 0.0001 0.3 0.2 256 0.3972299168975069]

...

[list([8, 4]) list([32]) False 0.001 0.4 0.2 256 0.47783933518005534]
[list([8, 4]) list([32]) False 0.001 0.4 0.3 128 0.5358725761772852]
[list([8, 4]) list([32]) False 0.001 0.4 0.3 256 0.4933518005540167]
[list([8, 4]) list([32]) False 0.001 0.4 0.4 128 0.4249307479224377]
[list([8, 4]) list([32]) False 0.001 0.4 0.4 256 0.5268698060941829]
[list([8, 4]) list([32]) False 0.0001 0.1 0.1 128 0.38753462603878114]
[list([8, 4]) list([32]) False 0.0001 0.1 0.1 256 0.44376731301939054]
[list([8, 4]) list([32]) False 0.0001 0.1 0.2 128 0.4789473684210527]
[list([8, 4]) list([32]) False 0.0001 0.1 0.2 256 0.4394736842105263]
[list([8, 4]) list([32]) False 0.0001 0.1 0.3 128 0.5473684210526315]
[list([8, 4]) list([32]) False 0.0001 0.1 0.3 256 0.46011080332409976]
[list([8, 4]) list([32]) False 0.0001 0.1 0.4 128 0.4714681440443213]
[list([8, 4]) list([32]) False 0.0001 0.1 0.4 256 0.4174515235457063]
[list([8, 4]) list([32]) False 0.0001 0.2 0.1 128 0.41274238227146814]
[list([8, 4]) list([

[list([8, 4]) list([32]) True 0.001 0.3 0.3 128 0.5232686980609419]
[list([8, 4]) list([32]) True 0.001 0.3 0.3 256 0.4481994459833795]
[list([8, 4]) list([32]) True 0.001 0.3 0.4 128 0.5047091412742383]
[list([8, 4]) list([32]) True 0.001 0.3 0.4 256 0.5354570637119113]
[list([8, 4]) list([32]) True 0.001 0.4 0.1 128 0.5875346260387811]
[list([8, 4]) list([32]) True 0.001 0.4 0.1 256 0.5002770083102493]
[list([8, 4]) list([32]) True 0.001 0.4 0.2 128 0.6260387811634349]
[list([8, 4]) list([32]) True 0.001 0.4 0.2 256 0.5473684210526316]
[list([8, 4]) list([32]) True 0.001 0.4 0.3 128 0.5177285318559557]
[list([8, 4]) list([32]) True 0.001 0.4 0.3 256 0.5556786703601108]
[list([8, 4]) list([32]) True 0.001 0.4 0.4 128 0.5631578947368421]
[list([8, 4]) list([32]) True 0.001 0.4 0.4 256 0.550415512465374]
[list([8, 4]) list([32]) True 0.0001 0.1 0.1 128 0.4229916897506925]
[list([8, 4]) list([32]) True 0.0001 0.1 0.1 256 0.4595567867036011]
[list([8, 4]) list([32]) True 0.0001 0.1 0.2 12

[list([8, 4]) list([16]) False 0.001 0.2 0.3 256 0.41024930747922445]
[list([8, 4]) list([16]) False 0.001 0.2 0.4 128 0.43656509695290857]
[list([8, 4]) list([16]) False 0.001 0.2 0.4 256 0.4795013850415513]
[list([8, 4]) list([16]) False 0.001 0.3 0.1 128 0.3814404432132964]
[list([8, 4]) list([16]) False 0.001 0.3 0.1 256 0.41246537396121885]
[list([8, 4]) list([16]) False 0.001 0.3 0.2 128 0.5468144044321329]
[list([8, 4]) list([16]) False 0.001 0.3 0.2 256 0.46980609418282554]
[list([8, 4]) list([16]) False 0.001 0.3 0.3 128 0.5060941828254848]
[list([8, 4]) list([16]) False 0.001 0.3 0.3 256 0.4581717451523546]
[list([8, 4]) list([16]) False 0.001 0.3 0.4 128 0.44238227146814396]
[list([8, 4]) list([16]) False 0.001 0.3 0.4 256 0.44155124653739614]
[list([8, 4]) list([16]) False 0.001 0.4 0.1 128 0.45678670360110807]
[list([8, 4]) list([16]) False 0.001 0.4 0.1 256 0.38116343490304705]
[list([8, 4]) list([16]) False 0.001 0.4 0.2 128 0.4795013850415513]
[list([8, 4]) list([16]) F

[list([8, 4]) list([16]) True 0.001 0.1 0.4 128 0.5030470914127424]
[list([8, 4]) list([16]) True 0.001 0.1 0.4 256 0.5207756232686982]
[list([8, 4]) list([16]) True 0.001 0.2 0.1 128 0.4937673130193906]
[list([8, 4]) list([16]) True 0.001 0.2 0.1 256 0.43878116343490303]
[list([8, 4]) list([16]) True 0.001 0.2 0.2 128 0.4889196675900277]
[list([8, 4]) list([16]) True 0.001 0.2 0.2 256 0.5563711911357341]
[list([8, 4]) list([16]) True 0.001 0.2 0.3 128 0.5]
[list([8, 4]) list([16]) True 0.001 0.2 0.3 256 0.5764542936288088]
[list([8, 4]) list([16]) True 0.001 0.2 0.4 128 0.5210526315789473]
[list([8, 4]) list([16]) True 0.001 0.2 0.4 256 0.5202216066481995]
[list([8, 4]) list([16]) True 0.001 0.3 0.1 128 0.5631578947368421]
[list([8, 4]) list([16]) True 0.001 0.3 0.1 256 0.585595567867036]
[list([8, 4]) list([16]) True 0.001 0.3 0.2 128 0.5587257617728533]
[list([8, 4]) list([16]) True 0.001 0.3 0.2 256 0.4867036011080333]
[list([8, 4]) list([16]) True 0.001 0.3 0.3 128 0.5681440443213

[list([8, 4]) list([8]) False 0.001 0.1 0.1 128 0.4146814404432133]
[list([8, 4]) list([8]) False 0.001 0.1 0.1 256 0.46149584487534623]
[list([8, 4]) list([8]) False 0.001 0.1 0.2 128 0.5322714681440444]
[list([8, 4]) list([8]) False 0.001 0.1 0.2 256 0.48711911357340726]
[list([8, 4]) list([8]) False 0.001 0.1 0.3 128 0.4867036011080332]
[list([8, 4]) list([8]) False 0.001 0.1 0.3 256 0.5235457063711911]
[list([8, 4]) list([8]) False 0.001 0.1 0.4 128 0.4277008310249307]
[list([8, 4]) list([8]) False 0.001 0.1 0.4 256 0.5520775623268698]
[list([8, 4]) list([8]) False 0.001 0.2 0.1 128 0.46675900277008314]
[list([8, 4]) list([8]) False 0.001 0.2 0.1 256 0.3945983379501385]
[list([8, 4]) list([8]) False 0.001 0.2 0.2 128 0.49085872576177286]
[list([8, 4]) list([8]) False 0.001 0.2 0.2 256 0.4433518005540166]
[list([8, 4]) list([8]) False 0.001 0.2 0.3 128 0.47534626038781164]
[list([8, 4]) list([8]) False 0.001 0.2 0.3 256 0.3933518005540166]
[list([8, 4]) list([8]) False 0.001 0.2 0.4

In [ ]:
deephit_cv_results = pd.read_csv("cv.results.deephit.csv")
print(deephit_cv_results["cindex"].values.max())
ind_best = deephit_cv_results["cindex"].values.argmax()

num_nodes_shared = eval(deephit_cv_results.iloc[ind_best, 0])
num_nodes_indiv = eval(deephit_cv_results.iloc[ind_best, 1])
batch_norm = deephit_cv_results.iloc[ind_best, 2]
lr = deephit_cv_results.iloc[ind_best, 3]

alpha = deephit_cv_results.iloc[ind_best, 4]
sigma = deephit_cv_results.iloc[ind_best, 5]
batch_size = deephit_cv_results.iloc[ind_best, 6]



net = CauseSpecificNet(in_features, num_nodes_shared, num_nodes_indiv, num_risks, out_features, batch_norm, dropout)
optimizer = tt.optim.AdamWR(lr = lr, decoupled_weight_decay = 0.01, cycle_eta_multiplier=0.8)
model = DeepHit(net, optimizer, alpha = alpha, sigma = sigma, duration_index = labtrans.cuts)
        

epochs = 100
callbacks = [tt.callbacks.EarlyStoppingCycle()]
log = model.fit(x_train, y_train, int(batch_size), epochs, callbacks, val_data = (x_test, y_test))

cif = model.predict_cif(x_test)
cif1 = pd.DataFrame(cif[0], model.duration_index)
ev = EvalSurv(1-cif1, durations_test, events_test == 1, censor_surv='km')

c_index = ev.concordance_td('antolini')
print('C-index: {:.4f}'.format(c_index))


time_grid = np.linspace(df_test["time"].values.min(), df_test["time"].values.max(), 100)
ibs = ev.integrated_brier_score(time_grid) 
print('IBS: {:.4f}'.format(ibs))


In [ ]:
def bootstrap_replicate_1d(data):
    bs_sample = np.random.choice(data,len(data))
    return bs_sample


bootstrap_R = 100
c_indexes = []
ibss = []


for i in range(bootstrap_R):
    print(i)
    train_bs_idx = bootstrap_replicate_1d(np.array(range(df_train.shape[0])))
    # Creating the X, T and E input
    X_train = x_train[train_bs_idx, ]
    T_train = y_train[0][train_bs_idx]
    E_train = y_train[1][train_bs_idx]
    Y_train = (T_train, E_train)
    net = CauseSpecificNet(in_features, num_nodes_shared, num_nodes_indiv, num_risks, out_features, batch_norm, dropout)
    optimizer = tt.optim.AdamWR(lr = lr, decoupled_weight_decay = 0.01, cycle_eta_multiplier=0.8)
    model = DeepHit(net, optimizer, alpha = alpha, sigma = sigma, duration_index = labtrans.cuts)
    epochs = 100
    callbacks = [tt.callbacks.EarlyStoppingCycle()]
    log = model.fit(x_train, y_train, int(batch_size), epochs, callbacks, val_data = (x_test, y_test))
    cif = model.predict_cif(x_test)
    cif1 = pd.DataFrame(cif[0], model.duration_index)
    ev = EvalSurv(1-cif1, durations_test, events_test == 1, censor_surv='km')
    # ev = EvalSurv(surv, Y_val_0, Y_val_1, censor_surv='km')
    c_index = ev.concordance_td('antolini')
    time_grid = np.linspace(df_test["time"].values.min(), df_test["time"].values.max(), 108)
    ibs = ev.integrated_brier_score(time_grid) 
    c_indexes.append(np.round(c_index, 4))
    ibss.append(np.round(ibs, 4))


pd.DataFrame(data = {"cindex": c_indexes, "ibs": ibss}).to_csv("results.ci.deephit.csv", index=False)

# Compute the 95% confidence interval: conf_int
mean_cindex = np.mean(c_indexes)
mean_ibs = np.mean(ibss)


# Print the mean
print('mean cindex =', mean_cindex)
print('mean ibs =', mean_ibs)


ci_cindex = np.percentile(c_indexes, [2.5, 97.5])
ci_ibs = np.percentile(ibss, [2.5, 97.5])

# Print the confidence interval
print('confidence interval =', ci_cindex)
print('confidence interval =', ci_ibs)